In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import datetime
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import telegram
from telegram import Bot
import asyncio
import schedule

current_time = datetime.datetime.now()
print(f"현재 시스템 시간: {current_time}")


# Signal.bz 뉴스 페이지에서 실시간 검색어 10가지를 가져오기
URL = "https://signal.bz/news"

# 텔레그램 설정
API_TOKEN = '7936899167:AAFl_3jTqQ-B2b4CUPaKnf_4tWgoY1FI_os'
CHAT_ID = '1059657134'

# 실시간 검색어 가져오기 함수 (Selenium 사용)
def get_realtime_search_terms(url):
    print(f"[{datetime.datetime.now()}] 실시간 검색어 수집을 시작합니다.")
    try:
        chrome_options = Options()
        chrome_options.add_argument("--headless")
        service = Service(ChromeDriverManager().install())
        driver = webdriver.Chrome(service=service, options=chrome_options)
        
        driver.get(url)
        print("웹페이지에 접속했습니다.")
        
        search_elements = driver.find_elements(By.XPATH, "/html/body/div[1]/div/main/div/section/div/section/section[1]/div[2]/div/div/div/a/span[2]")
        terms = [element.text.strip() for element in search_elements[:10]]
        
        driver.quit()
        if terms:
            print(f"[{datetime.datetime.now()}] 실시간 검색어 수집 성공: {terms}")
        else:
            print(f"[{datetime.datetime.now()}] 실시간 검색어를 수집하지 못했습니다.")
        return terms
    except Exception as e:
        print(f"[{datetime.datetime.now()}] 실시간 검색어 수집 중 오류 발생: {e}")
        return []

# 인기 뉴스 가져오기 함수 (Selenium 사용)
def get_popular_news(url):
    print(f"[{datetime.datetime.now()}] 인기 뉴스 수집을 시작합니다.")
    try:
        chrome_options = Options()
        chrome_options.add_argument("--headless")
        service = Service(ChromeDriverManager().install())
        driver = webdriver.Chrome(service=service, options=chrome_options)
        
        driver.get(url)
        print("웹페이지에 접속했습니다.")
        
        popular_news_elements = driver.find_elements(By.XPATH, "/html/body/div[1]/div/main/div/section/div/section/section[3]/div/section/div/div/div/div[2]")
        popular_news = [element.text.strip() for element in popular_news_elements[:10]]
        
        driver.quit()
        if popular_news:
            print(f"[{datetime.datetime.now()}] 인기 뉴스 수집 성공: {popular_news}")
        else:
            print(f"[{datetime.datetime.now()}] 인기 뉴스를 수집하지 못했습니다.")
        return popular_news
    except Exception as e:
        print(f"[{datetime.datetime.now()}] 인기 뉴스 수집 중 오류 발생: {e}")
        return []

# 검색어 출력 및 저장 함수
def save_search_terms(terms, popular_news):
    print(f"[{datetime.datetime.now()}] 검색어 및 인기 뉴스 저장을 시작합니다.")
    if terms:
        df = pd.DataFrame(terms, columns=['실시간 검색어'])
        current_date = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"실검top10_{current_date}.csv"
        df.to_csv(filename, index=False, encoding='utf-8-sig')
        print(f"검색어가 파일로 저장되었습니다: {filename}")
    else:
        print("검색어를 가져올 수 없습니다.")
    
    if popular_news:
        df_popular = pd.DataFrame(popular_news, columns=['인기 뉴스'])
        filename_popular = f"인기뉴스_{current_date}.csv"
        df_popular.to_csv(filename_popular, index=False, encoding='utf-8-sig')
        print(f"인기 뉴스가 파일로 저장되었습니다: {filename_popular}")
    else:
        print("인기 뉴스를 가져올 수 없습니다.")

# 텔레그램으로 메시지 전송 함수
async def send_message(api_token, chat_id, message):
    print(f"[{datetime.datetime.now()}] 텔레그램 메시지 전송을 시작합니다.")
    bot = Bot(token=api_token)
    try:
        await bot.send_message(chat_id=chat_id, text=message)
        print(f"[{datetime.datetime.now()}] 메시지가 성공적으로 전송되었습니다.")
    except telegram.error.TelegramError as e:
        print(f"[{datetime.datetime.now()}] 메시지 전송 실패: {e}")

# 주기적으로 실행하는 함수
def job():
    print(f"[{datetime.datetime.now()}] 주기적 작업 실행 시작")
    current_hour = datetime.datetime.now().hour
    if current_hour >= 1 or current_hour < 9:
        print(f"[{datetime.datetime.now()}] 현재는 메시지를 보내지 않는 시간입니다.")
        return
    
    realtime_terms = get_realtime_search_terms(URL)
    popular_news = get_popular_news(URL)
    save_search_terms(realtime_terms, popular_news)
    
    loop = asyncio.get_event_loop()
    
    if realtime_terms:
        message = "실시간 검색어:\n" + "\n".join(realtime_terms)
        loop.run_until_complete(send_message(API_TOKEN, CHAT_ID, message))
    
    if popular_news:
        message = "인기 뉴스:\n" + "\n".join(popular_news)
        loop.run_until_complete(send_message(API_TOKEN, CHAT_ID, message))

# 스케줄 설정 - 테스트를 위해 5분마다 실행
schedule.every(5).minutes.do(job)

if __name__ == "__main__":
    while True:
        print(f"[{datetime.datetime.now()}] 스케줄 확인 중...")
        schedule.run_pending()
        time.sleep(60)  # 1분마다 스케줄 확인


현재 시스템 시간: 2024-11-10 10:48:33.031813
[2024-11-10 10:48:33.033802] 스케줄 확인 중...
[2024-11-10 10:49:33.034166] 스케줄 확인 중...
[2024-11-10 10:50:33.035128] 스케줄 확인 중...
[2024-11-10 10:51:33.035442] 스케줄 확인 중...
[2024-11-10 10:52:33.035935] 스케줄 확인 중...
[2024-11-10 10:53:33.036342] 스케줄 확인 중...
[2024-11-10 10:53:33.036342] 주기적 작업 실행 시작
[2024-11-10 10:53:33.036342] 현재는 메시지를 보내지 않는 시간입니다.
[2024-11-10 10:54:33.036926] 스케줄 확인 중...
[2024-11-10 10:55:33.037210] 스케줄 확인 중...
[2024-11-10 10:56:33.037750] 스케줄 확인 중...
[2024-11-10 10:57:33.038401] 스케줄 확인 중...
[2024-11-10 10:58:33.039414] 스케줄 확인 중...
[2024-11-10 10:58:33.039414] 주기적 작업 실행 시작
[2024-11-10 10:58:33.039414] 현재는 메시지를 보내지 않는 시간입니다.
[2024-11-10 10:59:33.045042] 스케줄 확인 중...
[2024-11-10 11:00:33.046514] 스케줄 확인 중...
[2024-11-10 11:01:33.046768] 스케줄 확인 중...
[2024-11-10 11:02:33.047338] 스케줄 확인 중...
[2024-11-10 11:03:33.049948] 스케줄 확인 중...
[2024-11-10 11:03:33.050943] 주기적 작업 실행 시작
[2024-11-10 11:03:33.050943] 현재는 메시지를 보내지 않는 시간입니다.
[2024-11-10 11:04:33.051